In [1]:
from __future__ import print_function, division
import os
import torch
from numpy import newaxis
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from dataset_generator_2 import DataLoader,Dataset_sat
import h5py
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
from imgaug import augmenters as iaa
plt.ion()   # interactive mode
from image_utils import read_data_h5,standardize


/home/s161362/.conda/envs/env_dhi/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
PATH_INPUT='INPUT/'
PATH_OUTPUT='OUTPUT/'
NB_CLASSES=2


In [3]:
class Flip(object):
    """Flip ratio of the image Left/Right and up/down

    Args:
        ratio (int): how much of the image is flipped
    """

    def __init__(self,ratio):

        self.ratio = ratio

    def __call__(self, sample):
        X, Y = sample['input'], sample['groundtruth']
        seq = iaa.Sequential([iaa.Fliplr(0.5),iaa.Flipud(0.5)])
        data_tot=np.concatenate((X.astype('uint8'),Y.astype('uint8')),axis=2)
        data_tot=seq.augment_images(data_tot[newaxis,:,:,:])
        data_tot=np.squeeze(data_tot)
        X=data_tot[:,:,:X.shape[2]]
        Y=data_tot[:,:,X.shape[2]:]
        return {'input': X, 'groundtruth': Y}
    

In [4]:
def _parse_image(path_input,path_output,nb_classes):
    '''
    Reads and saves as as an array image input and output
    :paths_input path of the input image that have to be read  
    :paths_output path of the output image that have to be read  
    returns input and output image as array
    '''
    
    X=read_data_h5(path_input)
    Y_build=read_data_h5(path_output)
    inp=torch.LongTensor(Y_build)
    inp_ = torch.unsqueeze(inp, len(Y_build.shape))
    Y = torch.FloatTensor(Y_build.shape[0],Y_build.shape[1],nb_classes ).zero_()
    Y.scatter_(len(Y_build.shape), inp_, 1)
    Y=np.asarray(Y)
            
    return X,Y

In [5]:
class Dataset_sat(Dataset):
    """Satellite images dataset with rastered footprints in groundtruth."""

    def __init__(self,paths_input: np.ndarray,paths_output: np.ndarray,nb_classes: int,transform=None):
        """
        Args:
            
        """
        self.paths_input = paths_input
        self.paths_output = paths_output
        self.nb_classes=nb_classes
        self.transform = transform
    @classmethod
    def from_root_folder(cls, root_folder: str, nb_classes: int,*,transform=None, max_data_size:  int = None):
        paths_input = []
        paths_output=[]
        
        
        for filename in sorted(os.listdir(root_folder+PATH_INPUT))[:max_data_size]:
            paths_input.append(os.path.join(root_folder+PATH_INPUT, filename))

        for filename in sorted(os.listdir(root_folder+PATH_OUTPUT))[:max_data_size]:

            paths_output.append(os.path.join(root_folder+PATH_OUTPUT, filename))
        
        
        return Dataset_sat(np.asarray(paths_input), np.asarray(paths_output),nb_classes)

    def __len__(self):
        return len(self.paths_input)
    
#     def shuffled(self):
# #         if seed is not None:
# #             np.random.seed(seed)

#         idx = np.arange(len(self.paths_input))
#         np.random.shuffle(idx)
#         generator = Dataset_sat(self.paths_input[idx], self.paths_output[idx],self.nb_classes)

#         return generator 

    def __getitem__(self, idx):
        
        X,Y=_parse_image(self.paths_input[idx],self.paths_output[idx],self.nb_classes)     
        sample = {'input': X, 'groundtruth': Y}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [3]:
root_folder = '../2_DATA_GHANA/DATASET/120_x_120_8_pansh/TEST/'

# composed = transforms.Compose([Rescale(256),transform=Flip(0.5)
#                                RandomCrop(224)])
generator=Dataset_sat.from_root_folder(root_folder,NB_CLASSES)
    
dataloader = DataLoader(generator, batch_size=4,shuffle=True, num_workers=4)

# trsfrm=Flip(0.5)
# sample_batch=generator.__getitem__(4)
# plt.imshow(sample_batch['input'][:,:,0])
# plt.show()
# t_sample_batch = trsfrm(sample_batch)
# plt.imshow(t_sample_batch['input'][:,:,0])
# plt.show()
    

for i_batch,sample_batch in enumerate(dataloader):
    print( i_batch,sample_batch['input'].size(),
          sample_batch['groundtruth'].size())
    
    
    if i_batch==3:
        break
        

test2=standardize(sample_batch['input'])

0 torch.Size([4, 120, 120, 9]) torch.Size([4, 120, 120, 2])
1 torch.Size([4, 120, 120, 9]) torch.Size([4, 120, 120, 2])
2 torch.Size([4, 120, 120, 9]) torch.Size([4, 120, 120, 2])
3 torch.Size([4, 120, 120, 9]) torch.Size([4, 120, 120, 2])
